In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import pickle

import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [2]:
main_path=r'D:\python data\cv_flowers recognition\flowers'

In [3]:
categories=[]
for category in os.listdir(main_path):
    if os.path.isdir(os.path.join(main_path,category)):
        categories.append(category)
categories

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [4]:
X=[]
y=[]

for category in os.listdir(main_path):
    for img in tqdm(os.listdir(os.path.join(main_path,category))):
        img_path=os.path.join(main_path,category,img)
        label=categories.index(category)
        if img.split('.')[-1] not in ['jpeg','jpg','png','bmp']:
            os.remove(img_path)
            print(f'Removed {img}')
        elif img is not None:
            img_as_np=cv2.imread(img_path)
            img_as_np=cv2.resize(img_as_np,(256,256))/255

            X.append(img_as_np)
            y.append(label)

  0%|          | 0/764 [00:00<?, ?it/s]

100%|██████████| 984/984 [00:02<00:00, 428.18it/s]


In [5]:
X=np.array(X)
y=np.array(y)

In [6]:
combined = list(zip(X, y))
np.random.shuffle(combined)
shuffled_data, shuffled_labels = zip(*combined)

X = np.array(shuffled_data)
y = np.array(shuffled_labels)

batch_size = 32
X_batches = [X[i:i+batch_size] for i in range(0, len(X), batch_size)]

last_batch_length = len(X_batches[-1])
if last_batch_length < batch_size:
    padding_shape = (batch_size - last_batch_length, 256,256,3)
    X_batches[-1] = np.concatenate([X_batches[-1], np.zeros(padding_shape)], axis=0)

# Convert the list of batches into a NumPy array
X = np.array(X_batches)


batch_size = 32
y_batches = [y[i:i+batch_size] for i in range(0, len(y), batch_size)]

last_batch_length = len(y_batches[-1])
if last_batch_length < batch_size:
    padding_shape = (batch_size - last_batch_length,)
    y_batches[-1] = np.concatenate([y_batches[-1], np.zeros(padding_shape)], axis=0)

# Convert the list of batches into a NumPy array
y = np.array(y_batches)


#pickle.dump(X,open(os.path.join(os.path.dirname(main_path),'X.pickle'),'wb'))
#pickle.dump(y,open(os.path.join(os.path.dirname(main_path),'y.pickle'),'wb'))

X=pickle.load(open(os.path.join(os.path.dirname(main_path),'X.pickle'),'rb'))
y=pickle.load(open(os.path.join(os.path.dirname(main_path),'y.pickle'),'rb'))

In [7]:
X.shape,y.shape

((4317, 256, 256, 3), (4317,))

import random
batch_x=X[0]
batch_y=y[0]


fig,axes=plt.subplots(4,5,figsize=(20,14))
for i,index in enumerate(random.sample(range(len(batch_x)),20)):
    ax=axes.flatten()[i]
    ax.imshow(batch_x[index])
    ax.set_title(categories[int(batch_y[index])],size=12)
    ax.set_xticks([]), ax.set_yticks([])

In [8]:
train_index=int(len(X)*0.8)
val_index=int(len(X)*0.9)

X_train, y_train=X[:train_index], y[:train_index]
X_val, y_val=X[train_index:val_index], y[train_index:val_index]
X_test, y_test=X[val_index:], y[val_index:]

In [9]:
X_train.shape,X_val.shape,X_test.shape, y_train.shape,y_val.shape,y_test.shape

((3453, 256, 256, 3),
 (432, 256, 256, 3),
 (432, 256, 256, 3),
 (3453,),
 (432,),
 (432,))

In [10]:
from keras.models import Sequential
from keras.layers import Input, Conv2D,MaxPooling2D,MaxPooling3D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())


# Dense layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              #optimizer=Adam(learning_rate=0.001),  # Experiment with learning rate
              metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=5)

logdir='D:\python data\cv_flowers recognition\model\logs'
tensorboard_callbakcs=tf.keras.callbacks.TensorBoard(log_dir=logdir)


model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 64)      

In [12]:
hist=model.fit(X_train,y_train,epochs=15,validation_data=(X_val,y_val),batch_size=32)

Epoch 1/15


 64/108 [================>.............] - ETA: 43s - loss: 1.5819 - accuracy: 0.3848